In [31]:
import pandas as pd
import numpy as np
#from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, KNNBasic, NormalPredictor
from surprise import accuracy
from surprise.model_selection import KFold
from surprise.model_selection import train_test_split
import time
import datetime

# 数据处理
# read combined dataset #2
combined_data_2 = pd.read_csv('/Users/keweiding/Kaikeba_Exercise/combined_data_2.txt', sep=" ", header=None)
# 选取10%生成combined_data小数据集
combined_data_2_s = combined_data_2[:269823]
combined_data_2_s.rename(columns={0: 'content'}, inplace = True)
# 处理text数据集，将movie_id分配给相应的用户和评分，生成DataFrame
for x in combined_data_2_s.content:
    if ':' in x:
        index_start = combined_data_2_s.loc[combined_data_2_s['content'] == x, :].index[0]
        combined_data_2_s.loc[(index_start+1)::, 'movie_id'] = x.replace(':','')
        combined_data_2_s.drop(index_start,inplace=True)
# 拆分content项，为子项生成DataFrame列
combined_data_2_s[['user_id','score','date']] = combined_data_2_s['content'].str.split(',', expand = True)
combined_data_2_s.drop(columns='content', inplace = True)

combined_data_2_s['user_id'] = combined_data_2_s['user_id'].astype(int)
combined_data_2_s['score'] = combined_data_2_s['score'].astype(int)
combined_data_2_s['movie_id'] = combined_data_2_s['movie_id'].astype(int)
#combined_data_2_s = combined_data_2_s.set_index('user_id')
combined_data_2_s['date'] = combined_data_2_s['date'].apply(lambda x: x.replace('-','/'))
combined_data_2_s['date'] = combined_data_2_s['date'].apply(lambda x: time.mktime(datetime.datetime.strptime(x, "%Y/%m/%d").timetuple()))
combined_data_2_s.reset_index(inplace = True, drop = True)

# read probe dataset

probe_data = pd.read_csv('/Users/keweiding/Kaikeba_Exercise/probe.txt', sep=" ", header=None)
probe_data.rename(columns={0: 'content'}, inplace = True)
for x in probe_data.content:
    if ':' in x:
        index_start = probe_data.loc[probe_data['content'] == x, :].index[0]
        probe_data.loc[(index_start+1)::, 'movie_id'] = x.replace(':','')
        probe_data.drop(index_start,inplace=True)
probe_data.rename(columns={'content': 'user_id'}, inplace = True)

probe_data['user_id'] = probe_data['user_id'].astype(int)
#probe_data = probe_data.set_index('user_id')
probe_data['movie_id'] = probe_data['movie_id'].astype(int)
combined_data_2_s.reset_index(inplace = True, drop = True)

probe_data['probe'] = 1
df = pd.merge(combined_data_2_s,probe_data,how='left',on=['user_id','movie_id'])

new_df = df[~(df['probe']==1)][['user_id','movie_id','score','date']]
new_probe = df[df['probe']==1][['user_id','movie_id','score','date']]
new_df.shape[0],new_probe.shape[0]

new_probe.drop(columns='date', inplace=True)
new_df.drop(columns='date', inplace=True)

# 数据读取
reader = Reader(rating_scale=(1, 5))

train = Dataset.load_from_df(new_df,reader=reader)
test = Dataset.load_from_df(new_probe,reader=reader)

bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
algo = BaselineOnly(bsl_options=bsl_options)


train_probe, test_probe = train_test_split(test,test_size=.95)
algo.fit(train.build_full_trainset())
predictions = algo.test(test_probe)

# Then compute RMSE
accuracy.rmse(predictions)

/Users/keweiding/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/Users/keweiding/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/keweiding/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

Estimating biases using als...
RMSE: 1.0297


1.029740558981072

In [ ]:
#another way to slice data
